In [62]:
import sqlite3
import numpy as np
import pandas as pd
from datetime import datetime as dt, timedelta

In [63]:
airlines_df = pd.read_excel("../data/Airlines.xlsx", index_col="AirlineId")
airports_df = pd.read_excel("../data/Airports.xlsx", index_col="AirportId")

In [64]:
airports_df

,AirportCode,FullName,OriginCountry,OriginCity
AirportId,,,,
1,PDL,Ponta Delgada Joao Paulo II,Portugal,Ponta Delgada
2,FAO,Faro Airport,Portugal,Faro
3,LIS,Portela,Portugal,Lisbon
4,OPO,Franciso De Sa Carneiro,Portugal,Porto
5,SJZ,Sao Jorge Airport,Portugal,NaN
...,...,...,...,...
64,NCL,Newcastle International Airport,United Kingdom,Newcastle
65,LPL,Liverpool John Lennon Airport,United Kingdom,Liverpool
66,EDI,Edinburgh Airport,United Kingdom,Edinburgh


In [65]:
INSERT_TEMPLATE = "INSERT INTO {table_name} {columns_tuple}\n\tVALUES {values_tuple}"

In [66]:
print(INSERT_TEMPLATE.format(table_name="Airports", columns_tuple=tuple(airports_df.columns), values_tuple=tuple(airports_df.iloc[0].tolist())).replace("\'", ""))

INSERT INTO Airports (AirportCode, FullName, OriginCountry, OriginCity)
	VALUES (PDL, Ponta Delgada Joao Paulo II, Portugal, Ponta Delgada)


In [67]:
airlines_df

,Name,AirlineCode
AirlineId,,
1,Ryanair,RYR
2,Lufthansa Group,DLH
3,International Airlines Group,AAN
4,Air France-KLM,SEU
5,Easyjet,EZY
6,Wizz Air,WZZ
7,American Airlines,AAL
8,Delta Air Lines,DAL
9,Southwest Airlines,SWA


In [68]:
with open(r"../scripts/sql/populate_db.sql", "w", encoding="utf8") as f:
    for index in airlines_df.index.tolist():
        #print(INSERT_TEMPLATE.format(table_name="Airline", columns_tuple=tuple(airlines_df.columns), values_tuple=tuple(airlines_df.iloc[index - 1].tolist())).replace("\'", ""))
        f.write(INSERT_TEMPLATE.format(table_name="Airline", columns_tuple=tuple(airlines_df.columns), values_tuple=tuple(airlines_df.iloc[index -1].tolist())).replace("\'", ""))
        f.write("\n")
        
    for index in airports_df.index.tolist():
        f.write(INSERT_TEMPLATE.format(table_name="Airport", columns_tuple=tuple(airports_df.columns), values_tuple=tuple(airports_df.iloc[index - 1].tolist())).replace("\'", ""))
        f.write("\n")

In [69]:
def get_airline_info(name):
    airline = airlines_df.loc[airlines_df["Name"] == name].to_dict(orient="index")
    index = list(airline.keys())[0]
    
    return index, airline[index]["AirlineCode"]

def get_airport_info(city):
    airport = airports_df.loc[airports_df["OriginCity"] == city].to_dict(orient="index")
    index = list(airport.keys())[0]
    
    return index, airport[index]["AirportCode"]

In [95]:
def get_flight_info(departure, arrival, start, duration, airline, route_index):
    airline_id, airline_code = get_airline_info(airline)
    departure_id, departure_code = get_airport_info(departure)
    arrival_id, arrival_code = get_airport_info(arrival)
    
    return airline_id, route_index, airline_code + "001" + departure_code + arrival_code, duration, np.random.randint(50, 250), 0

In [96]:
def clock_to_timedetla(time):
    return timedelta(hours=int(time[0:2]), minutes=int(time[3:5]), seconds=int(time[6:8]))

In [97]:
def get_route_info(departure, arrival, start, duration, airline):
    airline_id, airline_code = get_airline_info(airline)
    departure_id, departure_code = get_airport_info(departure)
    arrival_id, arrival_code = get_airport_info(arrival)
    
    return departure_id, arrival_id, str(start), str(dt.strptime(start, "%Y-%m-%d %H:%M:%S") + clock_to_timedetla(duration))

In [99]:
get_flight_info("Rome", "Madrid", "2020-12-01 10:00:00", "02:20:00", "Ryanair", 1)

(1, 1, 'RYR001CIAMAD', '02:20:00', 233, 0)

In [100]:
date = dt.strptime("2020-12-01 10:00:00", "%Y-%m-%d %H:%M:%S")

In [101]:
get_route_info("Rome", "Madrid", "2020-12-01 10:00:00", "02:20:00", "Ryanair")

(18, 6, '2020-12-01 10:00:00', '2020-12-01 12:20:00')